# Genie Top200 Chart

In [1]:
import requests
import pandas as pd

### 1. 인터넷에서 데이터 가져오기

In [2]:
# Genie top200
url = 'https://www.genie.co.kr/chart/top200'
req = requests.get(url)         #  기본 html로 받을 것이므로 .jason 등 해줄 필요 x 
html = req.text                 # 이거 뭐? 
html 

# 잘못 가져온 거라 함 

'<br>\r\n<br>\r\n<center>\r\n<img src="http://www.geniemusic.co.kr/images/common/logo_r1.png"><br>\r\n<h2> <meta http-equiv="Content-Type" content="text/html;charset=UTF-8"> ì\xa0\x91ì\x86\x8dì\x9a\x94ì²\xadì\x9d´ ë³´ì\x95\x88ì\xa0\x95ì±\x85ì\x97\x90 ì\x9d\x98í\x95´ ì°¨ë\x8b¨ë\x90\x98ì\x97\x88ì\x8aµë\x8b\x88ë\x8b¤. ë\x8b¹ì\x82¬ ê³\xa0ê°\x9dì\x84¼í\x84°ë¡\x9c ë¬¸ì\x9d\x98í\x95´ì£¼ì\x8b\xadì\x8b\x9cì\x98¤.<br><br>\r\nThe security policy of the connection request is blocked. Contact your customer service representative.<br><br>\r\nì§\x80ë\x8b\x88ë®¤ì§\x81 ê³\xa0ê°\x9dì\x84¼í\x84° 1577-5337<br><br>\r\n</h2>\r\n</center>\r\n<br>'

In [3]:
# Chrome User-Agent
header = {'User-Agent':   
          "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36"
} 
req = requests.get(url, headers = header)
html = req.text
#html

# html 출력시키지 X

In [4]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

### 찾으려고 하는 데이터의 노드 찾기

- top 50 음악리스트만 찾고싶어

In [5]:
trs = soup.find_all('tr')
len(trs)

51

In [7]:
tbody = soup.select_one('table.list-wrap').find('tbody')
trs = tbody.find_all('tr')
len(trs)

50

In [10]:
tbody = soup.select_one('table.list-wrap').find('tbody')
trs = tbody.select('tr')
len(trs)

50

In [9]:
# tbody에서 우클릭 - copy selector)
# 개발자 도구에서 찾아주는 Selector 
tbody = soup.select_one('#body-content > div.newest-list > div > table > tbody')
trs = tbody.select('tr')
len(trs)

50

- Xpath 값은 아래와 같으나 BeautifulSoup은 지원하지 않음
<br>
//*[@id="body-content"]/div[6]/div/table/tbody

### 3. 여러개의 데이터 중 하나를 선택해서 원하는 정보를 추츨
#### rank, title, artist, album

- 첫 번째 데이터만

In [13]:
# 첫 번째 데이터
tr = trs[0]        

- rank

In [20]:
rank = tr.select_one('.number').get_text().split('\n')[0]
rank

'1'

- title

In [30]:
title = tr.select_one('.title.ellipsis').get_text().strip()     # strip : 앞뒤 공백 제거 (습관적으로 해주기)
title

'회전목마 (Feat. Zion.T & 원슈타인) (Prod. by Slom)'

- artist

In [37]:
artist = tr.select_one('.artist.ellipsis').get_text().strip()
artist

'sokodomo'

- album

In [48]:
album = tr.select_one('.albumtitle.ellipsis').get_text()
album

'쇼미더머니 10 Episode 2'

### 4. 한 페이지에 있는 모든 데이터를 반복문으로 가져와 데프 만들기

- 개별 리스트 만들기 ( rank_list, title_list...)
- '순위' : rank_list, ... 
- or 
- 딕셔너리의 리스트 만들기 
- {}

1. 반복문으로 리스트 생성 - 리스트로 데프 생성

In [52]:
rank_list, title_list, artist_list, album_list = [],[],[],[]        # 반복문 전 빈 리스트 생성
for tr in trs:
    
    rank = tr.select_one('.number').get_text().split('\n')[0]
    title = tr.select_one('.title.ellipsis').get_text().strip()
    artist = tr.select_one('.artist.ellipsis').get_text().strip()
    album = tr.select_one('.albumtitle.ellipsis').get_text()

    rank_list.append(rank)
    title_list.append(title)
    artist_list.append(artist)
    album_list.append(album)

df = pd.DataFrame({
    '순위' : rank_list, '제목' : title_list, '가수' : artist_list, '앨범' : album_list})
df.head()
    

,순위,제목,가수,앨범
0,1,회전목마 (Feat. Zion.T & 원슈타인) (Prod. by Slom),sokodomo,쇼미더머니 10 Episode 2
1,2,리무진 (Feat. MINO) (Prod. by GRAY),BE'O (비오),쇼미더머니 10 Episode 3
2,3,매일 크리스마스 (Everyday Christmas),다비치,매일 크리스마스 (Everyday Christmas)
3,4,만남은 쉽고 이별은 어려워 (Feat. Leellamarz) (Prod. by TOIL),베이식,쇼미더머니 10 Episode 3
4,5,불협화음 (Feat. AKMU) (Prod. by GRAY),Mudd the student,쇼미더머니 10 Semi Final


In [54]:
lines = []
for tr in trs:

    rank = tr.select_one('.number').get_text().split('\n')[0]
    title = tr.select_one('.title.ellipsis').get_text().strip()
    artist = tr.select_one('.artist.ellipsis').get_text().strip()
    album = tr.select_one('.albumtitle.ellipsis').get_text()

    lines.append([rank, title, artist, album])

df = pd.DataFrame(lines, columns = ['순위','제목','가수','앨범'])
df.head()
    

,순위,제목,가수,앨범
0,1,회전목마 (Feat. Zion.T & 원슈타인) (Prod. by Slom),sokodomo,쇼미더머니 10 Episode 2
1,2,리무진 (Feat. MINO) (Prod. by GRAY),BE'O (비오),쇼미더머니 10 Episode 3
2,3,매일 크리스마스 (Everyday Christmas),다비치,매일 크리스마스 (Everyday Christmas)
3,4,만남은 쉽고 이별은 어려워 (Feat. Leellamarz) (Prod. by TOIL),베이식,쇼미더머니 10 Episode 3
4,5,불협화음 (Feat. AKMU) (Prod. by GRAY),Mudd the student,쇼미더머니 10 Semi Final


In [56]:
lines = []
for tr in trs:
    rank = tr.select_one('.number').get_text().split('\n')[0]
    title = tr.select_one('.title.ellipsis').get_text().strip()
    artist = tr.select_one('.artist.ellipsis').get_text().strip()
    album = tr.select_one('.albumtitle.ellipsis').get_text()
    d = {'순위' : rank , '제목' : title, '가수' : artist, '앨범' : album}
    lines.append(d)

df = pd.DataFrame(lines)
df.tail()

,순위,제목,가수,앨범
45,46,고백,멜로망스 (MeloMance),세 번째 '고백'
46,47,시간을 거슬러 (낮에 뜨는 달 X 케이윌),케이윌 (K.Will),시간을 거슬러 (낮에 뜨는 달 X 케이윌)
47,48,그대라는 사치,임영웅,사랑의 콜센타 PART62
48,49,찰나가 영원이 될 때 (The Eternal Moment),마크툽 (Maktub),Red Moon : Beyond The Light
49,50,잊었니,임영웅,사랑의 콜센타 PART60


### 5. 모든 페이지의 데이터 가져오기 

In [ ]:
# 두번째 페이지 url 
https://www.genie.co.kr/chart/top200
https://www.genie.co.kr/chart/top200?ditc=D&ymd=20211208&hh=16&rtm=Y&pg=2


In [59]:
lines = []                  # 전체 페이지 걸쳐서 line에 넣으므로 반복문 밖으로 빼주기

for page in range(1,5):     # 전체 4 page임 

    url = f'https://www.genie.co.kr/chart/top200?ditc=D&ymd=20211208&hh=16&rtm=Y&pg={page}'
    req = requests.get(url, headers = header)
    html = req.text
    soup = BeautifulSoup(html, 'html.parser')
    tbody = soup.select_one('table.list-wrap').find('tbody')
    trs = tbody.select('tr')

    for tr in trs:          # 반복문의 반복문!!
        rank = tr.select_one('.number').get_text().split('\n')[0]
        title = tr.select_one('.title.ellipsis').get_text().strip()
        artist = tr.select_one('.artist.ellipsis').get_text().strip()
        album = tr.select_one('.albumtitle.ellipsis').get_text()
        lines.append([rank, title, artist, album])

df = pd.DataFrame(lines, columns=['순위', '제목','가수','앨범'])
df.head()

    
    


,순위,제목,가수,앨범
0,1,회전목마 (Feat. Zion.T & 원슈타인) (Prod. by Slom),sokodomo,쇼미더머니 10 Episode 2
1,2,리무진 (Feat. MINO) (Prod. by GRAY),BE'O (비오),쇼미더머니 10 Episode 3
2,3,매일 크리스마스 (Everyday Christmas),다비치,매일 크리스마스 (Everyday Christmas)
3,4,만남은 쉽고 이별은 어려워 (Feat. Leellamarz) (Prod. by TOIL),베이식,쇼미더머니 10 Episode 3
4,5,불협화음 (Feat. AKMU) (Prod. by GRAY),Mudd the student,쇼미더머니 10 Semi Final


In [60]:
df.tail()

,순위,제목,가수,앨범
195,196,할무니,김호중,할무니
196,197,Ashes,Stellar,Ashes
197,198,I Wish,다비치,매일 크리스마스 (Everyday Christmas)
198,199,거리,어반자카파,"지금, 헤어지는 중입니다 OST Part.8"
199,200,전화 한 번 못하니,신예영,전화 한 번 못하니 (너에게 하고 싶은 말 X 신예영)


In [62]:
df.to_csv('지니차트2_top200(211208).csv', index = False)